In [9]:
import os
import numpy as np
import cantera as ct
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
dfs = []

for i in range(1, 9):

    base_path = 'flame_speed_ch4_20210531_102003'
    file_path = f'/RUN_{i:06}/CKSoln_Flame_SpeedC1.ckcsv'
    path = base_path + file_path
    
    df_read = pd.read_csv(path, skiprows=1, header=None)

    columns_long = df_read.iloc[:, 0] + df_read.iloc[:, 1] 

    columns = []
    for name in columns_long:
        columns.append(name.split('_Run')[0])

    df = pd.DataFrame(data=df_read.iloc[:, 2:].T.to_numpy(), columns=columns)
    
    dfs.append(df)

In [4]:
print(columns)

['Distance', 'Pressure', 'Mass_density', 'Axial_velocity', 'Cross-section_area', 'Net_heat_production_from_gas-phase_reactions', 'EINO', 'EINO2', 'EINOX', 'EICO', 'Unburned_fuel_fraction', 'Temperature', 'Mole_fraction_H2', 'Mole_fraction_H', 'Mole_fraction_O', 'Mole_fraction_O2', 'Mole_fraction_OH', 'Mole_fraction_H2O', 'Mole_fraction_HO2', 'Mole_fraction_H2O2', 'Mole_fraction_C', 'Mole_fraction_CH', 'Mole_fraction_CH2', 'Mole_fraction_CH2(S)', 'Mole_fraction_CH3', 'Mole_fraction_CH4', 'Mole_fraction_CO', 'Mole_fraction_CO2', 'Mole_fraction_HCO', 'Mole_fraction_CH2O', 'Mole_fraction_CH2OH', 'Mole_fraction_CH3O', 'Mole_fraction_CH3OH', 'Mole_fraction_C2H', 'Mole_fraction_C2H2', 'Mole_fraction_C2H3', 'Mole_fraction_C2H4', 'Mole_fraction_C2H5', 'Mole_fraction_C2H6', 'Mole_fraction_HCCO', 'Mole_fraction_CH2CO', 'Mole_fraction_HCCOH', 'Mole_fraction_N', 'Mole_fraction_NH', 'Mole_fraction_NH2', 'Mole_fraction_NH3', 'Mole_fraction_NNH', 'Mole_fraction_NO', 'Mole_fraction_NO2', 'Mole_fraction

# Pre-procces
get Mass Fraction

In [29]:
# Load Molecular Weight
df_species = pd.read_csv('Species_List_GRI.csv')
num_species = df_species.shape[0]
columns_X = 'Mole_fraction_' + df_species['Name'].to_numpy()
columns_Y = 'Mass_fraction_' + df_species['Name'].to_numpy()
molecular_weights = df_species['Molecular_Weight'].to_numpy()

dfs_Y = []

for df in dfs:

    # Prepare ndarray from dataframe
    mole_fractions = df[columns_X].to_numpy()
    mean_molecular_weight = df['Molecular_weight'].to_numpy()

    # Calculate mass fraction
    mass_fractions = np.divide(np.multiply(mole_fractions, molecular_weights).T, mean_molecular_weight).T
    df_Y = pd.DataFrame(mass_fractions, columns=columns_Y)

    dfs_Y.append(df_Y)

# Test to get binary diffusion coefficient
using cantera

In [64]:
T = df['Temperature'][0]
P = df['Pressure'][0]
X = df[columns_X].to_numpy()[0, :]
Y = df_Y.to_numpy()[0, :]
dYdx = np.gradient(df_Y, df['Distance'], axis=0)[0, :]
rho = df['Mass_density'][0]

In [14]:
gas = ct.Solution('gri30.yaml')
gas.TPX = T, P, X

In [56]:
D_ij = gas.binary_diff_coeffs # m2/s

In [57]:
kb = 1.380649e-23 # J/K
e = 1.602176634e-19 # C

mu_ij = (e/(kb*T))*D_ij # m2/V/s

In [62]:
sum = np.zeros(num_species)

for i in range(num_species):
    for j in range(num_species):
        
        if i == j:
            continue

        sum[i] += X[i]/mu_ij[i, j]

mu_im = np.divide(1.0, sum) # m2/V/s

In [65]:
electric_field = 5e5 # V/m
Je = rho*electric_field*np.multiply(mu_im, Y)

In [66]:
Je

array([ 222.09853903,  164.89085429,  720.26137351,  918.50418367,
        750.63795863,  535.81645731,  941.43847314,  963.22670016,
        515.21777071,  626.46455392,  440.24817325,  440.24817325,
        461.05552998,  489.50379329,  800.67449518,  924.44648673,
        595.91737742,  611.30615667,  633.35293048,  633.35293048,
        640.85996949,  567.95595229,  584.55828569,  601.10050232,
        601.28931287,  581.4822877 ,  596.45448646, 1389.6155596 ,
        742.01990821,  742.01990821,  570.40287647,  710.85728258,
        739.56655135,  490.65731307,  833.59931601,  850.8141239 ,
       1073.00748665,  919.55100286,  876.12510811,  751.04789584,
        541.22602308,  556.06559228, 1389.58718047,  902.72479399,
        902.72479399,  902.72479399,  885.96285299,  807.79250401,
       1082.04773164,  664.35786578,  676.69244258,  756.0841222 ,
        771.88404184])

# Calculate Diffusion Coefficient

In [ ]:
D_dash = D - mu*Y*E/dYdx